In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/argugpt/argugpt.csv
/kaggle/input/argugpt/machine-dev.csv
/kaggle/input/argugpt/machine-test.csv
/kaggle/input/argugpt/machine-train.csv
/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [2]:
import pandas as pd 
import numpy as np
import matplotlib as plt 
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
train_pr_L = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
train_L = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
test_L =  pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
train_AI_A = pd.read_csv('/kaggle/input/argugpt/machine-train.csv')
test_AI_A = pd.read_csv('/kaggle/input/argugpt/machine-test.csv')
dev_AI_A =  pd.read_csv('/kaggle/input/argugpt/machine-dev.csv')
data_AI_DE = pd.read_csv('/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv')
data_AI_DP = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')


In [3]:
train_L

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [4]:
data_AI_A = pd.concat([train_AI_A, test_AI_A, dev_AI_A], axis=0)

In [5]:
data_AI_A.drop(columns=['score_level', 'score', 'exam_type', 'temperature', 'model', 'prompt'], inplace=True)

In [6]:
data_AI_A['prompt_id'] = 0

In [7]:
data_AI_A['generated'] = 1

In [8]:
new_data = pd.concat([train_L, data_AI_A], axis=0)

In [9]:
new_data

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
345,gre_514,0,The notion that one must be forced to defend a...,1
346,gre_1107,0,I strongly agree with the statement that menta...,1
347,gre_1163,0,"In today’s world, where competition is highly ...",1
348,gre_1174,0,Education is one of the most powerful tools th...,1


In [10]:
data_HUM_DE = data_AI_DE.drop(columns=['instructions','source_text'])
data_NOT_HUM_DE = data_AI_DE.drop(columns=['instructions','text'])

In [11]:
data_HUM_DE['generated'] = 0

In [12]:
data_HUM_DE['prompt_id'] = 0

In [13]:
data_HUM_DE

,id,text,generated,prompt_id
0,6060D28C05B6,Some schools in United States ofter classes fr...,0,0
1,60623DB5DE7A,"Four-day work week, a remarkable idea to conse...",0,0
2,607A39D981DE,Students and their families should consider an...,0,0
3,60ACDFA1609E,Agree you will never grow if something beyond ...,0,0
4,60AE13D3F07B,I think our character traits are formed by inf...,0,0
...,...,...,...,...
2416,F5FF5E9E553C,"On september 2, 2015 Generic_Name was entering...",0,0
2417,F60545D8271E,I think that schools must have a after school ...,0,0
2418,F610B3CBF3DF,"Winston Churchill once said ""success consists ...",0,0
2419,F610C7BCD9EC,Technology seems to be becoming more and more ...,0,0


In [14]:
data_HUM_DE = data_HUM_DE[['id', 'prompt_id', 'text', 'generated']]

In [15]:
data_NOT_HUM_DE

,id,source_text
0,6060D28C05B6,\nWhen considering the pros and cons of attend...
1,60623DB5DE7A,\nOne of the primary arguments for implementin...
2,607A39D981DE,\nBefore making any decisions about getting in...
3,60ACDFA1609E,"\nRalph Waldo Emerson once said, ""Go confident..."
4,60AE13D3F07B,\nHuman character traits are shaped by a wide ...
...,...,...
2416,F5FF5E9E553C,\nBecoming a surgeon requires a great deal of ...
2417,F60545D8271E,\nSchools should offer an after school homewor...
2418,F610B3CBF3DF,\nIt’s human nature to be afraid to make mista...
2419,F610C7BCD9EC,\nOne of the main debates of 2020 for many stu...


In [16]:
data_NOT_HUM_DE['generated'] = 1

In [17]:
data_NOT_HUM_DE['prompt_id'] = 0

In [18]:
data_NOT_HUM_DE = data_NOT_HUM_DE[['id', 'prompt_id', 'source_text', 'generated']]

In [19]:
data_NOT_HUM_DE.rename(columns={'source_text' : 'text'}, inplace=True)

In [20]:
data_NOT_HUM_DE

,id,prompt_id,text,generated
0,6060D28C05B6,0,\nWhen considering the pros and cons of attend...,1
1,60623DB5DE7A,0,\nOne of the primary arguments for implementin...,1
2,607A39D981DE,0,\nBefore making any decisions about getting in...,1
3,60ACDFA1609E,0,"\nRalph Waldo Emerson once said, ""Go confident...",1
4,60AE13D3F07B,0,\nHuman character traits are shaped by a wide ...,1
...,...,...,...,...
2416,F5FF5E9E553C,0,\nBecoming a surgeon requires a great deal of ...,1
2417,F60545D8271E,0,\nSchools should offer an after school homewor...,1
2418,F610B3CBF3DF,0,\nIt’s human nature to be afraid to make mista...,1
2419,F610C7BCD9EC,0,\nOne of the main debates of 2020 for many stu...,1


In [21]:
new_data_2 = pd.concat([new_data, data_NOT_HUM_DE, data_HUM_DE], axis=0)

In [22]:
new_data_2

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
2416,F5FF5E9E553C,0,"On september 2, 2015 Generic_Name was entering...",0
2417,F60545D8271E,0,I think that schools must have a after school ...,0
2418,F610B3CBF3DF,0,"Winston Churchill once said ""success consists ...",0
2419,F610C7BCD9EC,0,Technology seems to be becoming more and more ...,0


In [23]:
data_AI_DP['prompt'] = 0

In [24]:
data_AI_DP.drop(columns=['fold', 'source', 'label'], inplace=True)

In [25]:
data_AI_DP['generated'] = 1

In [26]:
data_AI_DP.rename(columns={'essay_id':'id', 'prompt':'prompt_id'}, inplace=True)

In [27]:
data_AI_DP = data_AI_DP[['id', 'prompt_id', 'text', 'generated']]

In [28]:
data_AI_DP

,id,prompt_id,text,generated
0,E897534557AF,0,"In recent years, technology has had a profoun...",1
1,DFBA34FFE11D,0,Should students participate in an extracurricu...,1
2,af37ecf5,0,The electoral college is a symbol of mockery a...,1
3,5EC2696BAD78,0,This is why I think the principle should allow...,1
4,llama_70b_v1843,0,I strongly believe that meditation and mindful...,1
...,...,...,...,...
44201,F7341069C4A4,0,"""Oh man I didn't make the soccer team!"", yelle...",1
44202,AFE6E553DAC2,0,I believe that using this technology could be ...,1
44203,falcon_180b_v1_600,0,The Face on Mars is a fascinating phenomenon t...,1
44204,A5F84C104693,0,Texting & Driving\n\nUsing your phone while dr...,1


In [29]:
new_data_3 = pd.concat([new_data_2, data_AI_DP], axis=0)

In [30]:
new_data_3

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
44201,F7341069C4A4,0,"""Oh man I didn't make the soccer team!"", yelle...",1
44202,AFE6E553DAC2,0,I believe that using this technology could be ...,1
44203,falcon_180b_v1_600,0,The Face on Mars is a fascinating phenomenon t...,1
44204,A5F84C104693,0,Texting & Driving\n\nUsing your phone while dr...,1


In [31]:
new_data_3['id'] = new_data_3.index

In [32]:
new_data_3

,id,prompt_id,text,generated
0,0,0,Cars. Cars have been around since they became ...,0
1,1,0,Transportation is a large necessity in most co...,0
2,2,0,"""America's love affair with it's vehicles seem...",0
3,3,0,How often do you ride in a car? Do you drive a...,0
4,4,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
44201,44201,0,"""Oh man I didn't make the soccer team!"", yelle...",1
44202,44202,0,I believe that using this technology could be ...,1
44203,44203,0,The Face on Mars is a fascinating phenomenon t...,1
44204,44204,0,Texting & Driving\n\nUsing your phone while dr...,1


In [33]:
new_data_3['id'] = new_data_3['id'].astype(dtype='object')

In [34]:
new_data_3['prompt_id'] = 0

In [35]:
new_data_3

,id,prompt_id,text,generated
0,0,0,Cars. Cars have been around since they became ...,0
1,1,0,Transportation is a large necessity in most co...,0
2,2,0,"""America's love affair with it's vehicles seem...",0
3,3,0,How often do you ride in a car? Do you drive a...,0
4,4,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
44201,44201,0,"""Oh man I didn't make the soccer team!"", yelle...",1
44202,44202,0,I believe that using this technology could be ...,1
44203,44203,0,The Face on Mars is a fascinating phenomenon t...,1
44204,44204,0,Texting & Driving\n\nUsing your phone while dr...,1


In [36]:
train, valid = train_test_split(new_data_3, test_size = 0.20, random_state=5, stratify=new_data_3.generated)

In [37]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3), sublinear_tf=True, max_features=10)

In [38]:
X_train = vectorizer.fit_transform(train['text'])
X_valid = vectorizer.transform(valid['text'])


In [39]:
classifier = XGBClassifier(objective='binary:logistic', learning_rate=0.01, n_estimators=1000, n_jobs=-1)
classifier.fit(X_train, train['generated'])

y_pred = classifier.predict_proba(X_valid)[:,1]

In [40]:
print("Validation AUC score:", roc_auc_score(valid["generated"], y_pred))

Validation AUC score: 0.6923578462307322


In [41]:
X_test = vectorizer.transform(test_L['text'])
y_test_pred = classifier.predict_proba(X_test)[:,1]

In [42]:
test_L['generated'] = y_test_pred
test_L[['id', 'generated']].to_csv('submission.csv', index=False)